<a href="https://colab.research.google.com/github/Theophilus03/RAG_Financial_Bot/blob/main/RAG_Financial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#streamlit

In [1]:
!pip install -q streamlit
%pip install -q google-colab-selenium
!pip install -qU "langchain-chroma>=0.1.2"
!pip install -q torch transformers accelerate bitsandbytes langchain langchain-community
!pip install -qU langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 33.7 MB/s eta 0:00:

In [15]:
%%writefile app.py
#selenium
import google_colab_selenium as gs
driver = gs.Chrome()
#web scraping
import time
import pandas as pd
from bs4 import BeautifulSoup

from selenium.webdriver.common.by import By

import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as ticker

#streamlit
import streamlit as st

#rag library
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain.docstore.document import Document

from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

import pandas as pd
import json


#Functions
def get_table():
  #beautiful soup
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  table = soup.find('div', class_='table yf-1pgoo1f')

  #get headers
  headers = table.find('div', class_='tableHeader')
  headers = headers.find_all('div', class_='column')



  headers_list = []
  for header in headers:
    headers_list.append(header.text)

  #data
  data = table.find('div', class_='tableBody')
  rows = data.find_all('div', class_='row')

  data = []
  for row in rows:
    cols = row.find_all('div', class_='column')
    cols = [col.text for col in cols]
    data.append(cols)

  #create dataframe and return it
  df = pd.DataFrame(data, columns=headers_list)

  return df


def WebScraping(url):
   #anual
  driver.get(url)
  expand_button = driver.find_element(By.CLASS_NAME, 'link2-btn')
  expand_button.click()

  time.sleep(1)
  df1 = get_table()

  #quarter
  quarterly_button = driver.find_element(By.ID, 'tab-quarterly')
  quarterly_button.click()
  time.sleep(2)

  df2 = get_table()
  return df1, df2


def table_transform(df):
  table = df
  table = table.transpose()
  col = table.iloc[0].str.strip()
  table = table[1:]
  table.columns = col
  table.replace('-- ', 0, inplace=True)
  table = table.replace(',', '', regex=True).astype(float)
  table = table.iloc[::-1]

  return table

def format_numbers(x, pos):
    sign = '-' if x < 0 else ''
    x = abs(x)

    if x >= 1e12:
        return f'{sign}{x*1e-12:.1f}T'
    elif x >= 1e9:
        return f'{sign}{x*1e-9:.1f}B'
    elif x >= 1e6:
        return f'{sign}{x*1e-6:.1f}M'
    else:
        return f'{sign}{x:.0f}'

def scrap_profile(url):
  driver.get(url)
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  desc = soup.find('section', {'data-testid': 'description'}, class_='yf-1hj9jti')
  p_tag = desc.find('p')
  text = p_tag.get_text(strip=True)
  return text



#Main
def main():
    #gemini embedding
    key = '######################################' #API KEY
    embeddings = GoogleGenerativeAIEmbeddings(google_api_key=key, model="models/embedding-001")
    #define LLM
    llm = ChatGoogleGenerativeAI(google_api_key=key, temperature=0.01, model="gemini-1.5-pro")

    #Streamlit
    st.set_page_config(page_title="Analisa Laporan Keuangan", page_icon=":money_with_wings:")
    st.header("Chatbot Laporan Keuangan  :money_with_wings:")


    with st.sidebar:
        st.subheader("Masukan Tiker Perusahaan")
        tiker = st.text_input("untuk idx tambahkan '.JK' contoh(BBCA.JK)")
        process = st.button("Process")


    if process and tiker:
        # Web scraping
        if f"{tiker}_IS" not in st.session_state:
            # Web scraping
            st.session_state[f"{tiker}_IS"] = WebScraping(f'https://finance.yahoo.com/quote/{tiker}/financials/')
            st.session_state[f"{tiker}_BS"] = WebScraping(f'https://finance.yahoo.com/quote/{tiker}/balance-sheet/')
            st.session_state[f"{tiker}_CF"] = WebScraping(f'https://finance.yahoo.com/quote/{tiker}/cash-flow/')
            st.session_state[f"{tiker}_desc"] = scrap_profile(f'https://finance.yahoo.com/quote/{tiker}/profile/')
        else:
            st.write(f"Data for {tiker} already scraped. Using cached data.")

        # Get the scraped data from session state
        IS = st.session_state[f"{tiker}_IS"]
        BS = st.session_state[f"{tiker}_BS"]
        CF = st.session_state[f"{tiker}_CF"]
        desc = st.session_state[f"{tiker}_desc"]

        #income Statement
        IS_final = table_transform(IS[0])
        zero_percentage = (IS_final == 0).sum(axis=1) / IS_final.shape[0]
        threshold = 0.7
        IS_final = IS_final.loc[zero_percentage < threshold]


        #Balance Sheets
        BS_annual = table_transform(BS[0])

        if IS_final.index[-1]=='TTM ':
          BS_quarterly = table_transform(BS[1])
          last_row = BS_quarterly.tail(1)
          BS_annual = pd.concat([BS_annual, last_row])
        zero_percentage = (BS_annual == 0).sum(axis=1) / BS_annual.shape[1]
        threshold = 0.7
        BS_final = BS_annual.loc[zero_percentage < threshold]

        #Cash Flow
        CF_final = table_transform(CF[0])

        zero_percentage = (CF_final == 0).sum(axis=1) / CF_final.shape[0]
        threshold = 0.7
        CF_final = CF_final.loc[zero_percentage < threshold]



        st.markdown(f"<h1 style='text-align: center;'>{tiker}</h1>", unsafe_allow_html=True)

        #Data Visualization
        st.markdown(f"<h3 style='text-align: center; text-decoration: underline;'>Income Statement</h3>", unsafe_allow_html=True)
        #IS
        npm = IS_final['Net Income Common Stockholders']/IS_final['Total Revenue']*100
        net_profit = IS_final['Net Income Common Stockholders']
        revenue = IS_final['Total Revenue']
        ebitda = IS_final['EBITDA']/IS_final['Total Revenue']*100


        #plot 1
        categories = IS_final.index
        x = np.arange(len(categories))
        width = 0.2  # Width of the bars

        # Create figure and axis objects
        fig, ax1 = plt.subplots(figsize=(10, 6))

        # Plotting Net Profit and Revenue as bar charts
        ax1.bar(x - width/2, net_profit, width=width, label='Net Profit', color='royalblue', align='center')
        ax1.bar(x + width/2, revenue, width=width, label='Revenue', color='goldenrod', align='center')

        # Setting X-axis labels
        ax1.set_xticks(x)
        ax1.set_xticklabels(categories)
        ax1.set_ylabel('(Rupiah)', fontsize=12)

        # Apply the custom formatter to the Y-axis
        ax1.yaxis.set_major_formatter(ticker.FuncFormatter(format_numbers))


        min_value = min(min(net_profit), min(revenue))
        max_value = max(max(net_profit), max(revenue))
        ax1.set_ylim([min_value*1.2, max_value*1.2])


        # Adding legends
        fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.9))

        ax1.grid(True, axis='y', linestyle='-', linewidth=0.5, color='gray')
        ax1.set_axisbelow(True)
        # Show plot
        st.pyplot(fig)


        #plot 2
        categories = IS_final.index
        x = np.arange(len(categories))

        # Create figure and axis objects
        fig, ax1 = plt.subplots(figsize=(10, 6))

        # Plotting Net Profit and Revenue as bar charts
        ax1.plot(x, npm, color='olivedrab', marker='o', label='Net Profit Margin')
        ax1.plot(x, ebitda, color='royalblue', marker='o', label='EBITDA Margin')

        #set limit
        min_val = min(min(npm), min(ebitda))
        max_val = max(max(npm), max(ebitda))
        ax1.set_ylim([min_val*1.2, max_val*1.2])

        # Setting X-axis labels
        ax1.set_xticks(x)
        ax1.set_xticklabels(categories)
        ax1.set_ylabel('(%)', fontsize=12)

        # Adding legends
        fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.9))

        # Show plot
        st.pyplot(fig)



        st.markdown(f"<h3 style='text-align: center; text-decoration: underline;'>Balance Sheets</h3>", unsafe_allow_html=True)
        #Balance sheets
        liabilities_long = BS_final['Current Liabilities']
        liabilities_short = BS_final['Total Non Current Liabilities Net Minority Interest']
        equity = BS_final['Total Equity Gross Minority Interest']
        assets = equity + liabilities_long + liabilities_short
        non_currect_assets = BS_final['Total non-current assets']
        DER = (liabilities_long + liabilities_short)/equity*100

        #plot 3
        categories = BS_final.index
        x = np.arange(len(categories))
        width = 0.2  # Width of the bars

        # Create figure and axis objects
        fig, ax1 = plt.subplots(figsize=(10, 6))

        # Plotting Net Profit and Revenue as bar charts
        ax1.bar(x - width, equity, width=width, label='Equity', color='royalblue', align='center')
        ax1.bar(x, liabilities_long, width=width, label='liabilities(Long Term)', color='red', align='center')
        ax1.bar(x, liabilities_short, width=width, label='liabilities(Short Tem)', color='darkred', align='center', bottom=liabilities_long)
        ax1.bar(x + width, assets, width=width, label='Assets', color='goldenrod', align='center')

        # Setting X-axis labels
        ax1.set_xticks(x)
        ax1.set_xticklabels(categories)
        ax1.set_ylabel('(Rupiah)', fontsize=12)

        # Apply the custom formatter to the Y-axis
        ax1.yaxis.set_major_formatter(ticker.FuncFormatter(format_numbers))

        # Adding legends
        fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.9))

        ax1.grid(True, axis='y', linestyle='-', linewidth=0.5, color='gray')
        ax1.set_axisbelow(True)

        # Show plot
        st.pyplot(fig)


        #plot 4
        categories = BS_final.index
        x = np.arange(len(categories))

        # Create figure and axis objects
        fig, ax1 = plt.subplots(figsize=(10, 6))

        # Plotting Net Profit and Revenue as bar charts
        ax1.plot(x, DER, color='olivedrab', marker='o', label='Debt to Equity Ratio')

        # Setting X-axis labels
        ax1.set_xticks(x)
        ax1.set_xticklabels(categories)
        ax1.set_ylabel('(%)', fontsize=12)

        # Adding legends
        fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.9))

        ax1.grid(True, axis='y', linestyle='-', linewidth=0.5, color='gray')
        ax1.set_axisbelow(True)
        # Show plot
        st.pyplot(fig)



        st.markdown(f"<h3 style='text-align: center; text-decoration: underline;'>Cash Flow</h3>", unsafe_allow_html=True)
        #Cash Flow
        operating = CF_final['Cash Flows from Used in Operating Activities Direct']
        investing = CF_final['Investing Cash Flow']
        financing = CF_final['Financing Cash Flow']
        beginning_cash = CF_final['Beginning Cash Position']
        cash_reserve = beginning_cash + operating + investing + financing
        end_cash = CF_final['End Cash Position']

        #plot 5
        categories = CF_final.index
        x = np.arange(len(categories))

        # Create figure and axis objects
        fig, ax1 = plt.subplots(figsize=(10, 6))

        # Plotting Net Profit and Revenue as bar charts
        ax1.plot(x, operating, color='olivedrab', marker='o', label='operating cash flow')
        ax1.plot(x, investing, color='royalblue', marker='o', label='investing cash flow')
        ax1.plot(x, financing, color='darkred', marker='o', label='financing cash flow')

        # Setting X-axis labels
        ax1.set_xticks(x)
        ax1.set_xticklabels(categories)
        ax1.set_ylabel('(Rupiah)', fontsize=12)
        ax1.yaxis.set_major_formatter(ticker.FuncFormatter(format_numbers))

        # Adding legends
        fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.9))

        ax1.grid(True, axis='y', linestyle='-', linewidth=0.5, color='gray')
        ax1.set_axisbelow(True)

        # Show plot
        st.pyplot(fig)



        #Store in Vector DB
        income_statement_json = IS_final.to_json()
        cash_flow_json = BS_final.to_json()
        balance_sheet_json = CF_final.to_json()
        combined_data = {
            "company_description": desc,
            "income_statement": json.loads(income_statement_json),
            "cash_flow": json.loads(cash_flow_json),
            "balance_sheet": json.loads(balance_sheet_json)
        }
        document = Document(page_content=json.dumps(combined_data))

        st.session_state.db = Chroma.from_documents([document], embeddings)

        #retrieval RAG for full analysis
        prompt_template = """
        You are a financial analyst. Based on the company profile and the financial reports provided, your task is to provide a detailed financial analysis of the company, focusing on significant increase  or decrease  with a threshold of 10%.
        Use only the financial reports provided in the context below. Do not make assumptions or hallucinate information.

        {context}

        Use the following structure to guide your analysis:
        **Company Profile:**
        - Describe the complete company profile from company_description.

        1. **Income Statement Analysis:**
          - **Revenue Growth:**
            - Detail the number and percentage changes and the periods they occurred. example "2020-2021: Increased by 14.2% (Rp 1,9 to Rp 2,5)"

          - **Gross Profit:**
            - Detail the number and percentage changes and the periods they occurred.

          - **Net Income:**
            - Detail the number and percentage changes and the periods they occurred.

        2. **Cash Flow Statement Analysis:**
          - **Operating Cash Flow:**
            - Detail the number and percentage changes and the periods they occurred.

          - **Investing Cash Flow:**
            - Detail the number and percentage changes and the periods they occurred.
          - **Financing Cash Flow:**
            - Detail the number and percentage changes and the periods they occurred.

        3. **Balance Sheet Analysis:**
          - **Total Assets:**
            - Detail the number and percentage changes and the periods they occurred.
          - **Equity:**
            - Detail the number and percentage changes and the periods they occurred.
          - **Debt-to-Equity Ratio:**
            - Detail the number and percentage changes and the periods they occurred.

        4. **Summary:**
          - Summarize the overall financial performance for each period.

        Now, answer the query for this company: {question}
        """

        prompt = ChatPromptTemplate.from_template(prompt_template)

        qa_with_sources = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=st.session_state.db.as_retriever(),
            chain_type_kwargs={"prompt": prompt},
            return_source_documents=True,
        )

        #ask the bot to analyze it
        analysis = qa_with_sources("Please provide an overall financial analysis")
        report = f"{analysis['result']}"
        st.markdown(report)



    #QnA with the bot
    question = st.text_input("Tanyakan tentang laporan keuangan yang diupload:")
    if question:
      if 'db' not in st.session_state:
        st.session_state.db = None

      if st.session_state.db:
          # Define the prompt template
          concise_prompt_template = """
          You are a financial analyst. Provide a concise answer to the following question based on the financial data provided and provide the company name.

          {context}

          Now, answer the query: {question}
          """
          concise_prompt = ChatPromptTemplate.from_template(concise_prompt_template)

          qa_with_sources_concise = RetrievalQA.from_chain_type(
              llm=llm,
              chain_type="stuff",
              retriever=st.session_state.db.as_retriever(),
              chain_type_kwargs={"prompt": concise_prompt},
              return_source_documents=True,
          )

          # Get the answer
          answer = qa_with_sources_concise(question)
          st.markdown(f"**Jawaban:** {answer['result']}")
      else:
          st.markdown("Harap masukan tiker dan proses terlebih dahulu")

    #quit selenium browser
    if st.button('Quit Driver'):
      driver.quit()
      st.success("WebDriver has been quit.")
if __name__ == '__main__':
    main()




Overwriting app.py


In [16]:
!wget -q -O - ipv4.icanhazip.com

34.87.162.233


In [17]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.87.162.233:8501

your url is: https://honest-boxes-lick.loca.lt
<IPython.core.display.HTML object>
Hit:1 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jam